In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bokeh')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

In [ ]:
df = pd.read_csv('/content/dataset.csv')
df

In [ ]:
df.isna().sum()

In [ ]:
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],format='%d-%m-%Y %H:%M:%S')

df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
df["OccupancyRate"] = df["Occupancy"] / df["Capacity"]
df

In [ ]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "QueueLength", "TrafficConditionNearby", "IsSpecialDay", "VehicleType"]].to_csv("parking_stream.csv", index=False)

In [ ]:
import pathway as pw

In [ ]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber: str
    LastUpdatedDate: str
    LastUpdatedTime: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: str
    VehicleType: str

In [ ]:
input_table = pw.io.csv.read(
    "dataset.csv",
    schema=ParkingSchema,
    mode="static",
)

In [ ]:
traffic_map = {"Low": 1, "Medium": 2, "High": 3}
vehicle_weights = {"car": 0.1, "bike": 0.05, "truck": 0.2}
BASE_PRICE = 10.0
α, β, γ, δ, λ = 0.5, 0.3, 0.2, 0.4, 0.6

cleaned = input_table.select(
    input_table.SystemCodeNumber,
    input_table.LastUpdatedDate,
    input_table.LastUpdatedTime,
    occupancy_rate=pw.apply(lambda o, c: o / c if c else 0, input_table.Occupancy, input_table.Capacity),
    queue=input_table.QueueLength,
    traffic_num=pw.apply(lambda x: traffic_map.get(x, 2), input_table.TrafficConditionNearby),
    is_special=pw.apply(lambda x: 1 if x.lower() == "yes" else 0, input_table.IsSpecialDay),
    vehicle_weight=pw.apply(lambda x: vehicle_weights.get(x.lower(), 0), input_table.VehicleType)
)

with_demand = cleaned.select(
    cleaned.SystemCodeNumber,
    cleaned.LastUpdatedDate,
    cleaned.LastUpdatedTime,
    cleaned.occupancy_rate,
    cleaned.queue,
    cleaned.traffic_num,
    cleaned.is_special,
    cleaned.vehicle_weight,
    demand=pw.apply(
        lambda o, q, t, s, v: α * o + β * q - γ * t + δ * s + v,
        cleaned.occupancy_rate, cleaned.queue, cleaned.traffic_num, cleaned.is_special, cleaned.vehicle_weight
    )
)

final = with_demand.select(
    with_demand.SystemCodeNumber,
    with_demand.LastUpdatedDate,
    with_demand.LastUpdatedTime,
    with_demand.demand,
    norm_demand=pw.apply(lambda d: (d - 0.0) / (3.0 - 0.0 + 1e-6), with_demand.demand),
    price=pw.apply(lambda d: max(min(BASE_PRICE * (1 + λ * ((d - 0.0) / (3.0 - 0.0 + 1e-6))), 2 * BASE_PRICE), 0.5 * BASE_PRICE),
                  with_demand.demand)
)

In [ ]:
pw.io.csv.write(final, filename="model2_stream_output.csv")
pw.run()

In [ ]:
print(df.columns.tolist())

In [ ]:
import pandas as pd

df_streamed = pd.read_csv("model2_stream_output.csv")

df_streamed["timestamp"] = pd.to_datetime(
    df_streamed["LastUpdatedDate"] + " " + df_streamed["LastUpdatedTime"],
    errors="coerce",
    dayfirst=True
)

df_streamed["date"] = df_streamed["timestamp"].dt.date
df_streamed["lot"] = df_streamed["SystemCodeNumber"]

daily_avg = df_streamed.groupby(["lot", "date"])["price"].mean().reset_index()
daily_avg["date_str"] = daily_avg["date"].astype(str)

In [ ]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show, output_notebook
output_notebook()

plots = []

for lot_id in df_streamed["SystemCodeNumber"].unique():
    lot_df = df_streamed[df_streamed["SystemCodeNumber"] == lot_id].copy()
    lot_df = lot_df.sort_values("timestamp")
    source = ColumnDataSource(lot_df)

    p = figure(
        title=f"Model 2 Price Trend – Lot: {lot_id}",
        x_axis_type="datetime",
        width=900,
        height=300,
        tools="pan,box_zoom,reset,save"
    )

    p.line(x="timestamp", y="price", source=source, line_width=2, color="steelblue", legend_label="Price")
    p.circle(x="timestamp", y="price", source=source, size=4, color="steelblue")

    hover = HoverTool(
        tooltips=[
            ("Timestamp", "@timestamp{%F %H:%M}"),
            ("Price", "@price{0.00}")
        ],
        formatters={"@timestamp": "datetime"},
        mode="vline"
    )
    p.add_tools(hover)

    p.xaxis.axis_label = "Time"
    p.yaxis.axis_label = "Price"
    p.legend.visible = False
    p.xaxis.major_label_orientation = 0.8

    plots.append(p)

# Display all plots in a scrollable column
show(column(*plots))